In [21]:
import os
import openai
from llama_index.vector_stores.neo4jvector import Neo4jVectorStore
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from dotenv import load_dotenv
load_dotenv()


True

In [27]:
username = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')
url = os.getenv("NEO4J_URI")
print(url)
embed_dim = 1536
# username = "neo4j"
# password = "pleaseletmein"
# url = "bolt://localhost:7687"


neo4j+s://e8c3c851.databases.neo4j.io


In [47]:

neo4j_vector = Neo4jVectorStore(username, password, url, embed_dim)
# load documents
documents = SimpleDirectoryReader(input_dir='./data_store/translations').load_data()
print(documents)

[Document(id_='7f9b7a18-cfdd-407b-aa27-537228127686', embedding=None, metadata={'file_path': 'c:\\Users\\Sagar G R\\LittleBear\\Graph-RAG\\graph_rag\\data_store\\translations\\littlebear_translations.txt', 'file_name': 'littlebear_translations.txt', 'file_type': 'text/plain', 'file_size': 5696, 'creation_date': '2024-10-23', 'last_modified_date': '2024-10-23'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='"full_text":"Hannover (energate) - A survey among business representatives recently painted an ambivalent picture of Germany as a production location. Although many companies appreciate the stable framework conditions in Germany for their production, at the same time concerns are growing about rising energy costs and increasing bureaucracy. energate 

In [48]:
from llama_index.core import StorageContext

storage_context = StorageContext.from_defaults(vector_store=neo4j_vector)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (row) { ... }} {position: line: 1, column: 21, offset: 20} for query: "UNWIND $data AS row CALL { WITH row MERGE (c:`Chunk` {id: row.id}) WITH c, row CALL db.create.setVectorProperty(c, 'embedding', row.embedding) YIELD node SET c.`text` = row.text SET c += row.metadata } IN TRANSACTIONS OF 1000 ROWS"
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('db.create.setVectorProperty' has been replaced by 'db.create.setNodeVectorProperty')} {position: lin

In [49]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import PropertyGraphIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor

from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

In [107]:
graph_store = Neo4jPropertyGraphStore(username, password, url, )
# load documents
documents = SimpleDirectoryReader(input_dir='./data_store/translations').load_data()
print(documents)

[Document(id_='3618c3a5-a099-4d8b-89f7-eb1518f71b3e', embedding=None, metadata={'file_path': 'c:\\Users\\Sagar G R\\LittleBear\\Graph-RAG\\graph_rag\\data_store\\translations\\littlebear_translations.txt', 'file_name': 'littlebear_translations.txt', 'file_type': 'text/plain', 'file_size': 17192, 'creation_date': '2024-10-23', 'last_modified_date': '2024-10-23'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='"full_text":"Hannover (energate) - A survey among business representatives recently painted an ambivalent picture of Germany as a production location. Although many companies appreciate the stable framework conditions in Germany for their production, at the same time concerns are growing about rising energy costs and increasing bureaucracy. energate

In [99]:
from typing import Literal
entities = Literal[
    "PERSON", 
    "ORGANISATION", 
    "LOCATION", 
    "TECHNOLOGY", 
    "ECONOMIC_FACTOR", 
    "EVENT", 
    "SURVEY", 
    "TOPIC", 
    "PROJECT", 
    "RESOURCE", 
    "GOAL"
]

relations = Literal[
    "WORKS_FOR", 
    "HAS_SUBSIDIARY", 
    "MERGES_WITH", 
    "PRODUCES", 
    "LOCATED_IN", 
    "PARTICIPATES_IN", 
    "CONDUCTS", 
    "HOSTS", 
    "PARTICIPATED_BY", 
    "MENTIONS", 
    "USED_BY", 
    "AFFECTS", 
    "IMPACTS", 
    "PARTNER_OF", 
    "USES_TECHNOLOGY", 
    "SUPPLIES_RESOURCE", 
    "HAS_PROJECT", 
    "HAS_GOAL", 
    "INVESTS_IN", 
    "DECOMMISSIONED"
]

schema = {
    "PERSON": [
        "WORKS_FOR",      # (Person)-[:WORKS_FOR]->(Organisation)
        "PARTICIPATES_IN",# (Person)-[:PARTICIPATES_IN]->(Event)
        "CONDUCTS"        # (Person)-[:CONDUCTS]->(Survey)
    ],
    "ORGANISATION": [
        "HAS_SUBSIDIARY",  # (Organisation)-[:HAS_SUBSIDIARY]->(Organisation)
        "MERGES_WITH",     # (Organisation)-[:MERGES_WITH]->(Organisation)
        "PRODUCES",        # (Organisation)-[:PRODUCES]->(Resource/Technology)
        "LOCATED_IN",      # (Organisation)-[:LOCATED_IN]->(Location)
        "HOSTS",           # (Organisation)-[:HOSTS]->(Event)
        "USED_BY",         # (Organisation)-[:USED_BY]->(Technology)
        "AFFECTS",         # (Organisation)-[:AFFECTS]->(EconomicFactor)
        "IMPACTS",         # (Organisation)-[:IMPACTS]->(EconomicFactor)
        "PARTNER_OF",      # (Organisation)-[:PARTNER_OF]->(Organisation)
        "USES_TECHNOLOGY", # (Organisation)-[:USES_TECHNOLOGY]->(Technology)
        "SUPPLIES_RESOURCE",# (Organisation)-[:SUPPLIES_RESOURCE]->(Resource)
        "HAS_PROJECT",     # (Organisation)-[:HAS_PROJECT]->(Project)
        "HAS_GOAL",        # (Organisation)-[:HAS_GOAL]->(Goal)
        "INVESTS_IN",      # (Organisation)-[:INVESTS_IN]->(Project)
        "DECOMMISSIONED"   # (Organisation)-[:DECOMMISSIONED]->(Facility)
    ],
    "LOCATION": [
        "LOCATED_IN"       # (Entity)-[:LOCATED_IN]->(Location) (e.g., Organisation, Project)
    ],
    "TECHNOLOGY": [
        "USED_BY",         # (Technology)-[:USED_BY]->(Organisation)
        "PRODUCES",        # (Technology)-[:PRODUCES]->(Resource)
        "USES_TECHNOLOGY"  # (Technology)-[:USES_TECHNOLOGY]->(Organisation/Project)
    ],
    "ECONOMIC_FACTOR": [
        "AFFECTS",         # (EconomicFactor)-[:AFFECTS]->(Entity)
        "IMPACTS"          # (EconomicFactor)-[:IMPACTS]->(Entity)
    ],
    "EVENT": [
        "PARTICIPATES_IN", # (Entity)-[:PARTICIPATES_IN]->(Event)
        "HOSTS",           # (Event)-[:HOSTS]->(Organisation)
        "PARTICIPATED_BY"  # (Event)-[:PARTICIPATED_BY]->(Person)
    ],
    "SURVEY": [
        "MENTIONS",        # (Survey)-[:MENTIONS]->(Topic)
        "CONDUCTED_BY"     # (Survey)-[:CONDUCTED_BY]->(Person/Organisation)
    ],
    "TOPIC": [
        "MENTIONS"         # (Entity)-[:MENTIONS]->(Topic)
    ],
    "PROJECT": [
        "LOCATED_IN",      # (Project)-[:LOCATED_IN]->(Location)
        "USES_TECHNOLOGY", # (Project)-[:USES_TECHNOLOGY]->(Technology)
        "HAS_GOAL",        # (Project)-[:HAS_GOAL]->(Goal)
        "INVESTS_IN"       # (Organisation)-[:INVESTS_IN]->(Project)
    ],
    "RESOURCE": [
        "SUPPLIES_RESOURCE",# (Resource)-[:SUPPLIES_RESOURCE]->(Organisation)
        "PRODUCES_RESOURCE" # (Resource)-[:PRODUCES_RESOURCE]->(Organisation)
    ],
    "GOAL": [
        "HAS_GOAL"         # (Entity)-[:HAS_GOAL]->(Goal)
    ]
}


In [103]:

# recommended uppercase, underscore separated
# entities = Literal["PERSON","ORGANISATIONS","LOCATIONS","TECHNOLOGIES","ECONOMIC_FACTORS","EVENTS"]
# relations = Literal["PART_OF", "HAS", "IS_A",]
# schema = {
#     "PERSON": ["PART_OF", "HAS", "IS_A"],
#     "PLACE": ["PART_OF", "HAS"],
#     "THING": ["IS_A"],
# }

schema_kg_extractor = SchemaLLMPathExtractor(
    llm=OpenAI(model="gpt-4o-mini"),
    possible_entities=entities,
    possible_relations=relations,
    kg_validation_schema=schema,
    strict=False,  # if false, will allow triplets outside of the schema
    num_workers=4,
    max_triplets_per_chunk=10,
)

In [104]:
import nest_asyncio 
nest_asyncio.apply()
# Extract graph from documents
index = PropertyGraphIndex.from_documents(
    documents,
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
    kg_extractors=[
       schema_kg_extractor
    ],
    property_graph_store=graph_store,
    show_progress=True,
)

Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.08it/s]


In [108]:
# Define retriever
retriever = index.as_retriever(
    include_text=True,  # include source text in returned nodes, default True
)
results = retriever.retrieve("what are some of the concerns discussed")
for record in results:
    print(record.text)

Here are some facts extracted from the provided text:

Christian Otto -> PARTICIPATES_IN -> survey among Germany's companies
Climate Economy Foundation -> PARTNER_OF -> Christian Otto
Christian Otto -> PARTICIPATES_IN -> Association of Energy Consumers (VEA)
MAN Energy Solutions -> HAS_SUBSIDIARY -> VW Group
MAN Energy Solutions -> HAS_SUBSIDIARY -> Quest One

"full_text":"Hannover (energate) - A survey among business representatives recently painted an ambivalent picture of Germany as a production location. Although many companies appreciate the stable framework conditions in Germany for their production, at the same time concerns are growing about rising energy costs and increasing bureaucracy. energate spoke to co-initiator Christian Otto, Managing Director of the Association of Energy Consumers (VEA), about the results of the study. energate: Mr. Otto, you recently conducted a survey among Germany's companies together with the Climate Economy Foundation, which paints a sobering pic

In [71]:
query_engine = index.as_query_engine(include_text=True,similarity_top_k=10)
response = query_engine.query("what were the main comments by Otto")
print(str(response))

Companies are facing high and volatile energy costs, bureaucratic hurdles, and political uncertainty. The survey conducted among Germany's companies highlighted concerns about the recent rise in production costs, weakening competitiveness, and potential job losses in industries. Otto emphasized the need for politicians to address solvable problems, reduce bureaucracy, and ensure economically viable energy costs to support companies investing in climate-friendly technologies.


In [112]:
from llama_index.core.query_engine import CitationQueryEngine

query_engine = CitationQueryEngine.from_args(
    index,
    # increase the citation chunk size!
    citation_chunk_size=1024,
    similarity_top_k=10,
)
response = query_engine.query("who is olaf and andreas. have they met before? if yes where did they meet?")
print(str(response))

print(response.source_nodes[0].node.get_text())

Olaf Scholz is the Federal Chancellor who attended the opening ceremony of Quest One's series production of electrolysis stacks in Hamburg-Rahlstedt [2]. Andreas Görtz is the President Sustainable Power Solutions at Rolls-Royce Power Systems [1]. There is no information provided in the sources about whether Olaf Scholz and Andreas Görtz have met before or where they might have met.
Source 1:
Here are some facts extracted from the provided text:

Andreas Görtz -> WORKS_FOR -> Rolls-Royce Power Systems
Quest One -> WORKS_FOR -> Robin von Plettenberg
Robin von Plettenberg -> WORKS_FOR -> Quest One
Stawag - Stadt- und Städteregionswerke Aachen AG -> HAS_GOAL -> advance the energy and heat transition
Rolls-Royce -> MERGES_WITH -> Hoeller
Rolls-Royce -> DECOMMISSIONED -> Hoeller
Quest One -> PRODUCES -> Hamburg-Rahlstedt
Aachen -> LOCATED_IN -> Stawag

The CEO of Quest One, Robin von Plettenberg, is optimistic about the future utilization of production: "The hydrogen economy will change mass